In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors, LocalOutlierFactor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
import nfl_data_py as nfl
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
today = dt.date.today()
year = today.year
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [34]:
df = nfl.import_schedules(years=range(year-3,year))

In [35]:
# Prepare dataframe by dropping irrelevant predictors and formatting columns for KNN
df['Under'] = np.where(df['total'] < df['total_line'], 1, 0)
df['Push'] = np.where(df['total'] == df['total_line'], 1, 0)
df = df[df.Push != 1]

def date_to_month(time_str):
    year, month, day = map(int, time_str.split('-'))
    return month
df['month'] = df['gameday'].apply(date_to_month)
# Function to convert time to seconds
def time_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60
# Apply the function to the 'time' column
df['gametime'] = df['gametime'].apply(time_to_seconds)

dict_day = {"weekday": {"Sunday": 0, "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6}}
df.replace(dict_day, inplace=True)
dict_roof = {"roof": {"outdoors": 0, "dome": 1, "closed": 2, "open": 3}}
df.replace(dict_roof, inplace=True)
dict_surface = {"surface": {"grass": 0, "grass ": 0, "fieldturf": 1, "astroturf": 2, "sportturf": 3, "matrixturf": 4, "astroplay": 5, "a_turf": 6, "dessograss": 7}}
df.replace(dict_surface, inplace=True)
dict_surface = {"surface": {"grass": 0, "grass ": 0, "fieldturf": 1, "astroturf": 2, "sportturf": 3, "matrixturf": 4, "astroplay": 5, "a_turf": 6, "dessograss": 7}}
df.replace(dict_surface, inplace=True)

df = pd.get_dummies(df, drop_first=True, columns=['game_type', 'location', 'stadium_id', 'home_team', 'away_team'])
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [36]:
df.columns
features = df.drop(['Under', 'Push', 'gameday', 'game_id', 'home_score', 'away_score', 'result', 'total', 'overtime', 'old_game_id', 'gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_qb_id', 'home_qb_id', 'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee', 'stadium', 'wind', 'temp'], axis=1).columns

# Accuracy Testing

In [37]:
# Model building
y = df.Under
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=9, stratify=y)

precis_array = []
acc_array = []
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
for i in range(1, 51):
    rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=i)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    acc = accuracy_score(y_test, preds)
    precis = precision_score(y_test, preds)
    precis_array.append(precis)
    acc_array.append(acc)
# combined = pd.DataFrame(dict(actual=y_test, prediction=preds))
# pd.crosstab(index=combined['actual'], columns=combined['prediction'])

In [38]:
def find_mean(array):
    if len(array) == 0:
        return 0  # To handle empty arrays
    return sum(array) / len(array)
find_mean(precis_array)

0.6682009739940774

In [39]:
find_mean(acc_array)

0.6222222222222228

# Weekly Plays

In [54]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import nfl_data_py as nfl
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
today = dt.date.today()
year = today.year

# Variable declaration
df = nfl.import_schedules(years=range(year-3,year+1))
currSeason = df[df.season == year]
predWeek = currSeason[['week', 'total']].dropna()
if np.isnan(predWeek.week.max()):
    predWeek = 1
else:
    predWeek = predWeek.week.max() + 1

predWeek=2
# Prepare dataframe by dropping irrelevant predictors and formatting columns for KNN
df['Under'] = np.where(df['total'] < df['total_line'], 1, 0)
df['Push'] = np.where(df['total'] == df['total_line'], 1, 0)
df = df[df.Push != 1]

def date_to_month(time_str):
    year, month, day = map(int, time_str.split('-'))
    return month
df['month'] = df['gameday'].apply(date_to_month)
# Function to convert time to seconds
def time_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60
# Apply the function to the 'time' column
df['gametime'] = df['gametime'].apply(time_to_seconds)

dict_day = {"weekday": {"Sunday": 0, "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6}}
df.replace(dict_day, inplace=True)
dict_roof = {"roof": {"outdoors": 0, "dome": 1, "closed": 2, "open": 3}}
df.replace(dict_roof, inplace=True)
dict_surface = {"surface": {"grass": 0, "grass ": 0, "fieldturf": 1, "astroturf": 2, "sportturf": 3, "matrixturf": 4, "astroplay": 5, "a_turf": 6, "dessograss": 7}}
df.replace(dict_surface, inplace=True)
dict_surface = {"surface": {"grass": 0, "grass ": 0, "fieldturf": 1, "astroturf": 2, "sportturf": 3, "matrixturf": 4, "astroplay": 5, "a_turf": 6, "dessograss": 7}}
df.replace(dict_surface, inplace=True)

df = pd.get_dummies(df, drop_first=True, columns=['game_type', 'location', 'stadium_id', 'home_team', 'away_team'])

features = df.drop(['Under', 'Push', 'gameday', 'game_id', 'home_score', 'away_score', 'result', 'total', 'overtime', 'old_game_id', 'gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_qb_id', 'home_qb_id', 'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee', 'stadium', 'wind', 'temp'], axis=1).columns

train_df = df[(df.season < year) | ((df.season == year) & (df.week < predWeek))]
test_df = df[(df.season == year) & (df.week == predWeek)]
train_df.dropna(inplace=True)
X_train = train_df[features]
y_train = train_df.Under
X_test = test_df[features]
y_test = test_df.Under

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
X_test['Prediction'] = preds
X_test['Prediction'] = np.where(X_test.Prediction == 1, "Under", "Over")
# Predicted Plays log
nextPlays = pd.merge(right=X_test, left=currSeason, right_index=True, left_index=True, how='left')
nextPlays = nextPlays[['game_id', 'season_x', 'week_x', 'home_team', 'away_team', 'gametime_x', 'weekday_x', 'total_line_x', 'Prediction', 'under_odds_x', 'over_odds_x']]
nextPlays.columns = ['Game ID', 'Season', 'Week', 'Home', 'Away', 'Start Time', 'Day', 'Total Line', 'Prediction', 'Under Odds', 'Over Odds']
nextPlays = nextPlays[nextPlays.Week == predWeek]
# Value cleanup
dict_day = {"Day": {0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}}
nextPlays.replace(dict_day, inplace=True)
nextPlays

,Game ID,Season,Week,Home,Away,Start Time,Day,Total Line,Prediction,Under Odds,Over Odds
6722,2024_02_BUF_MIA,2024,2,MIA,BUF,20:15,Thursday,48.5,Over,-105.0,-115.0
6723,2024_02_LV_BAL,2024,2,BAL,LV,13:00,Sunday,41.0,Over,-108.0,-112.0
6724,2024_02_LAC_CAR,2024,2,CAR,LAC,13:00,Sunday,39.0,Under,-112.0,-108.0
6725,2024_02_NO_DAL,2024,2,DAL,NO,13:00,Sunday,47.0,Under,-108.0,-112.0
6726,2024_02_TB_DET,2024,2,DET,TB,13:00,Sunday,51.5,Over,-112.0,-108.0
6727,2024_02_IND_GB,2024,2,GB,IND,13:00,Sunday,41.5,Under,-110.0,-110.0
6728,2024_02_CLE_JAX,2024,2,JAX,CLE,13:00,Sunday,41.5,Under,-108.0,-112.0
6729,2024_02_SF_MIN,2024,2,MIN,SF,13:00,Sunday,46.5,Under,-108.0,-112.0
6730,2024_02_SEA_NE,2024,2,NE,SEA,13:00,Sunday,39.0,Under,-110.0,-110.0
6731,2024_02_NYJ_TEN,2024,2,TEN,NYJ,13:00,Sunday,41.5,Under,-112.0,-108.0


In [53]:
nextPlays = pd.merge(right=X_test, left=currSeason, right_index=True, left_index=True, how='left')
nextPlays[nextPlays.week_x==2]

,game_id,season_x,game_type,week_x,gameday,weekday_x,gametime_x,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest_x,home_rest_x,away_moneyline_x,home_moneyline_x,spread_line_x,away_spread_odds_x,home_spread_odds_x,total_line_x,under_odds_x,over_odds_x,div_game_x,roof_x,surface_x,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,season_y,week_y,weekday_y,gametime_y,away_rest_y,home_rest_y,away_moneyline_y,home_moneyline_y,spread_line_y,away_spread_odds_y,home_spread_odds_y,total_line_y,under_odds_y,over_odds_y,div_game_y,roof_y,surface_y,month,game_type_DIV,game_type_REG,game_type_SB,game_type_WC,location_Neutral,stadium_id_BAL00,stadium_id_BOS00,stadium_id_BUF00,stadium_id_CAR00,stadium_id_CHI98,stadium_id_CIN00,stadium_id_CLE00,stadium_id_DAL00,stadium_id_DEN00,stadium_id_DET00,stadium_id_FRA00,stadium_id_GER00,stadium_id_GNB00,stadium_id_HOU00,stadium_id_IND00,stadium_id_JAX00,stadium_id_KAN00,stadium_id_LAX01,stadium_id_LON00,stadium_id_LON02,stadium_id_MEX00,stadium_id_MIA00,stadium_id_MIN01,stadium_id_NAS00,stadium_id_NOR00,stadium_id_NYC01,stadium_id_PHI00,stadium_id_PHO00,stadium_id_PIT00,stadium_id_SAO00,stadium_id_SEA00,stadium_id_SFO01,stadium_id_TAM00,stadium_id_VEG00,stadium_id_WAS00,home_team_ATL,home_team_BAL,home_team_BUF,home_team_CAR,home_team_CHI,home_team_CIN,home_team_CLE,home_team_DAL,home_team_DEN,home_team_DET,home_team_GB,home_team_HOU,home_team_IND,home_team_JAX,home_team_KC,home_team_LA,home_team_LAC,home_team_LV,home_team_MIA,home_team_MIN,home_team_NE,home_team_NO,home_team_NYG,home_team_NYJ,home_team_PHI,home_team_PIT,home_team_SEA,home_team_SF,home_team_TB,home_team_TEN,home_team_WAS,away_team_ATL,away_team_BAL,away_team_BUF,away_team_CAR,away_team_CHI,away_team_CIN,away_team_CLE,away_team_DAL,away_team_DEN,away_team_DET,away_team_GB,away_team_HOU,away_team_IND,away_team_JAX,away_team_KC,away_team_LA,away_team_LAC,away_team_LV,away_team_MIA,away_team_MIN,away_team_NE,away_team_NO,away_team_NYG,away_team_NYJ,away_team_PHI,away_team_PIT,away_team_SEA,away_team_SF,away_team_TB,away_team_TEN,away_team_WAS,Prediction
6722,2024_02_BUF_MIA,2024,REG,2,2024-09-12,Thursday,20:15,BUF,31.0,MIA,10.0,Home,-21.0,41.0,0.0,2024091200,59524,NaN,202409120mia,NaN,401671807,6465.0,4,4,114.0,-135.0,2.5,-110.0,-110.0,48.5,-105.0,-115.0,1,outdoors,grass,86.0,3.0,00-0034857,00-0036212,Josh Allen,Tua Tagovailoa,Sean McDermott,Mike McDaniel,Bill Vinovich,MIA00,Hard Rock Stadium,2024.0,2.0,4.0,72900.0,4.0,4.0,114.0,-135.0,2.5,-110.0,-110.0,48.5,-105.0,-115.0,1.0,0.0,0.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Over
6723,2024_02_LV_BAL,2024,REG,2,2024-09-15,Sunday,13:00,LV,26.0,BAL,23.0,Home,-3.0,49.0,0.0,2024091500,59525,NaN,202409150rav,NaN,401671624,6466.0,7,10,350.0,-455.0,9.0,-112.0,-108.0,41.0,-108.0,-112.0,0,outdoors,grass,76.0,10.0,00-0035289,00-0034796,Gardner Minshew,Lamar Jackson,Antonio Pierce,John Harbaugh,Tra Blake,BAL00,M&T Bank Stadium,2024.0,2.0,0.0,46800.0,7.0,10.0,350.0,-455.0,9.0,-112.0,-108.0,41.0,-108.0,-112.0,0.0,0.0,0.0,9.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Over
6724,2024_02_LAC_CAR,2024,REG,2,2024-09-15,Sunday,13:00,LAC,26.0,CAR,3.0,Ho

In [48]:
X_test

,season,week,weekday,gametime,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,month,game_type_DIV,game_type_REG,game_type_SB,game_type_WC,location_Neutral,stadium_id_BAL00,stadium_id_BOS00,stadium_id_BUF00,stadium_id_CAR00,stadium_id_CHI98,stadium_id_CIN00,stadium_id_CLE00,stadium_id_DAL00,stadium_id_DEN00,stadium_id_DET00,stadium_id_FRA00,stadium_id_GER00,stadium_id_GNB00,stadium_id_HOU00,stadium_id_IND00,stadium_id_JAX00,stadium_id_KAN00,stadium_id_LAX01,stadium_id_LON00,stadium_id_LON02,stadium_id_MEX00,stadium_id_MIA00,stadium_id_MIN01,stadium_id_NAS00,stadium_id_NOR00,stadium_id_NYC01,stadium_id_PHI00,stadium_id_PHO00,stadium_id_PIT00,stadium_id_SAO00,stadium_id_SEA00,stadium_id_SFO01,stadium_id_TAM00,stadium_id_VEG00,stadium_id_WAS00,home_team_ATL,home_team_BAL,home_team_BUF,home_team_CAR,home_team_CHI,home_team_CIN,home_team_CLE,home_team_DAL,home_team_DEN,home_team_DET,home_team_GB,home_team_HOU,home_team_IND,home_team_JAX,home_team_KC,home_team_LA,home_team_LAC,home_team_LV,home_team_MIA,home_team_MIN,home_team_NE,home_team_NO,home_team_NYG,home_team_NYJ,home_team_PHI,home_team_PIT,home_team_SEA,home_team_SF,home_team_TB,home_team_TEN,home_team_WAS,away_team_ATL,away_team_BAL,away_team_BUF,away_team_CAR,away_team_CHI,away_team_CIN,away_team_CLE,away_team_DAL,away_team_DEN,away_team_DET,away_team_GB,away_team_HOU,away_team_IND,away_team_JAX,away_team_KC,away_team_LA,away_team_LAC,away_team_LV,away_team_MIA,away_team_MIN,away_team_NE,away_team_NO,away_team_NYG,away_team_NYJ,away_team_PHI,away_team_PIT,away_team_SEA,away_team_SF,away_team_TB,away_team_TEN,away_team_WAS,Prediction
6722,2024,2,4,72900,4,4,114.0,-135.0,2.5,-110.0,-110.0,48.5,-105.0,-115.0,1,0,0.0,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Over
6723,2024,2,0,46800,7,10,350.0,-455.0,9.0,-112.0,-108.0,41.0,-108.0,-112.0,0,0,0.0,9,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Over
6724,2024,2,0,46800,7,7,-225.0,185.0,-5.0,-110.0,-110.0,39.0,-112.0,-108.0,0,0,0.0,9,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Under
6725,2024,2,0,46800,7,7,235.0,-290.0,6.5,-112.0,-108.0,47.0,-108.0,-112.0,0,2,4.0,9,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Under
6726,2024,2,0,46800,7,7,295.0,-375.0,7.5,-108.0,-112.0,51.5,-112.0,-108.0,0,1,1.0,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Over
6727,2024,2,0,46800,7,9,-142.0,120.0,-3.0,-102.0,-118.0,41.5,-110.0,-110.0,0,0,0.0,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Under
6728,2024,2,0,46800,7,7,142.0,-170.0,3.0,100.0,-120.0,41.5,-108.0,-112.0,0,0,0.0,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Under
6729,2024,2,0,46800,6,7,-218.0,180.0,-4.5,-112.0,-108.0,46.5,-108.0,-112.0,0,1,3.0,9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0

In [51]:
currSeason[currSeason.index == 6722]

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6722,2024_02_BUF_MIA,2024,REG,2,2024-09-12,Thursday,20:15,BUF,31.0,MIA,10.0,Home,-21.0,41.0,0.0,2024091200,59524,NaN,202409120mia,NaN,401671807,6465.0,4,4,114.0,-135.0,2.5,-110.0,-110.0,48.5,-105.0,-115.0,1,outdoors,grass,86.0,3.0,00-0034857,00-0036212,Josh Allen,Tua Tagovailoa,Sean McDermott,Mike McDaniel,Bill Vinovich,MIA00,Hard Rock Stadium
